In [1]:
# permutations that require max number of dcj operations  - diameter of an instance 

# find the largest value of n which you can find the diameter 
# n --> diameter (randomly sample x number)
# increase n  

In [2]:
using Random

using NBInclude
@nbinclude("dcj_algo.ipynb")

SRC (.,a:t)(a:h,.)(b:h,b:t) --> TARGET (a:t,a:h)(.,b:t)(b:h,.)
DCJ :: (.,a:t)(a:h,.) --> (a:t,a:h)(.,.)
DCJ :: (b:h,b:t) --> (b:t,.)(b:h,.)
#operations = 2
--------------------------------------------------------------------------------------


2

In [3]:
# Function to get the first n letters of the alphabet
function first_n_letters(n::Int)
    return collect('a':'z')[1:n]
end

first_n_letters (generic function with 1 method)

In [4]:
# generates genomes/permutations/strings of length n 
# (up to 26 bc alphabet)
function generate_genome_str(n::Int)
    # generate first n letters
    letters = first_n_letters(n)

    # randomize capitalization (gene orientation)
    for i in length(letters) 
        if rand(Bool)
            letters[i] = uppercase(letters[i])
        end
    end 

    # shuffle letters to create permutation 
    letters = shuffle(letters)

    # randomly choose length of chromosomes and circular/linear 
    max_chrom_length = n 
    curr_letters_idx = 1

    genome_str = ""
    while max_chrom_length != 0
        # choose rand length up to max_chrom_length & decremenet max_chrom_length 
        chrom_length = rand(1:max_chrom_length)
        chrom = ""

        max_chrom_length -= chrom_length

        # choose linear or circ 
        linear = rand(Bool)

        # if linear, add telomeres 
        end_idx = curr_letters_idx + chrom_length - 1
        chrom = letters[curr_letters_idx:end_idx]
        if linear 
            chrom = "." * join(chrom) * "."
        end 
        
        chrom = join(chrom)
        # add comma after the last element if max_chrom_length != 0 
        if max_chrom_length != 0 
            genome_str = genome_str * chrom * ","
        else
            genome_str = genome_str * chrom 
            # insert commas to separate chromosomes 
        end 
        
        curr_letters_idx += chrom_length 
    end 

    return genome_str
end

generate_genome_str(2)

".Ba."

In [6]:
function calculate_diameter(n::Int, target::String, num_src_permutations_to_generate::Int)
    max_dist = 0 
    src = generate_genome_str(n)
    max_dist_permutation = src

    for i in 1:num_src_permutations_to_generate
        
        dist = calculate_distance(src, target)
        if max_dist < dist 
            max_dist = dist
            max_dist_permutation = src 
        end 
        src = generate_genome_str(n)
    end 

    return max_dist, max_dist_permutation
end 


function calculate_diameters(n::Int, num_src_permutations_to_generate::Int) 
    diameters = Vector{Tuple{Int, String, String}}()

    for i in 1:n
        s = "GENOMES WITH " * string(i) * " GENE\n"
        printstyled(s; color = :red)

        target = generate_genome_str(i) 
        diam, max_dist_src = calculate_diameter(i, target, num_src_permutations_to_generate)

        push!(diameters, (diam, max_dist_src, target))  # i == idx
    end 

    return diameters
end 

n = 10
calculate_diameters(n, 5)

GENOMES WITH 1 GENE
SRC (a:h,a:t) --> TARGET (.,a:h)(a:t,.)
DCJ :: (a:h,a:t) --> (a:h,.)(a:t,.)
#operations = 1
--------------------------------------------------------------------------------------
SRC (.,a:h)(a:t,.) --> TARGET (.,a:h)(a:t,.)
#operations = 0
--------------------------------------------------------------------------------------
SRC (.,a:h)(a:t,.) --> TARGET (.,a:h)(a:t,.)
#operations = 0
--------------------------------------------------------------------------------------
SRC (.,a:t)(a:h,.) --> TARGET (.,a:h)(a:t,.)
#operations = 0
--------------------------------------------------------------------------------------
SRC (.,a:h)(a:t,.) --> TARGET (.,a:h)(a:t,.)
#operations = 0
--------------------------------------------------------------------------------------
GENOMES WITH 2 GENE
SRC (b:h,b:t)(a:h,a:t) --> TARGET (.,b:h)(b:t,.)(a:h,a:t)
DCJ :: (b:h,b:t) --> (b:h,.)(b:t,.)
#operations = 1
-------------------------------------------------------------------------------

10-element Vector{Tuple{Int64, String, String}}:
 (1, "a", ".A.")
 (2, ".a.,b", ".B.,a")
 (3, ".ab.,.c.", ".bCa.")
 (4, "dcb,a", ".cD.,.a.,b")
 (5, ".eb.,.dc.,.a.", "baec,d")
 (5, "aFcdbe", ".efba.,cd")
 (7, ".adcebf.,.g.", "cfabge,.d.")
 (8, "abdgfHc,.e.", ".dH.,bafegc")
 (9, "a,ei,.hgb.,.fd.,c", "decfgIhab")
 (9, "ijfdb,hc,.ega.", ".cgJfheabd.,.i.")